In [ ]:
using Plots
using Random
using Distributions

Random.seed!(1234);
rng = MersenneTwister(1234);
gr();

include("../kernel/kernel.jl")

In [ ]:
function create_toy_data(func, n_samples::Int64, std=1.0, domain=[0.0, 1.0])
    x = collect(range(domain[1], domain[2], length=n_samples));
    shuffle!(rng, x);
    noise = rand(Uniform(-std, std), n_samples);
    return x, func.(x) + noise
end

function sinusoidal(x)
    return sin.(2 * pi .* x)
end

## 6.1 Dual Representation

In [ ]:
x_train, y_train = create_toy_data(sinusoidal, 10, 0.2);

x_train = reshape(x_train, 1, size(x_train)[1])
model = GaussianProcessRegressor(PolynomialKernel(3, 1.0), 1e10)
fitting(model, x_train, y_train);

x = reshape(collect(range(0.0, 1.0, length=100)), 1, 100);
y, y_sigmas = predict(model, x, true)

plot(x[1, :], y[1, :], lw=2, color="green", label="regression")
plot!(x_train[1, :], y_train, seriestype="scatter", label="training")
plot!(x[1, :], sin.(2 * pi .* x[1, :]), lw=2, color="red", label="true")

## 6.4 Gaussian Process

### 6.4.2 Gaussian process for regression

In [ ]:
x_train, y_train = create_toy_data(sinusoidal, 7, 0.1, [0.0, 0.7]);
x_train = reshape(x_train, 1, size(x_train)[1])
model = GaussianProcessRegressor(RBFKernel([1.0, 15.0]), 100);
fitting(model, x_train, y_train)

x = reshape(collect(range(0.0, 1.0, length=100)), 1, 100);
y, y_sigmas = predict(model, x, true)

y_std = sqrt.(reshape(y_sigmas, length(y_sigmas)))
plot(x[1, :], y[1, :], ribbon=(y_std, y_std), color="green", label="std")
plot!(x_train[1, :], y_train, seriestype="scatter", label="training")
plot!(x[1, :], sin.(2 * pi .* x[1, :]), lw=2, color="red", label="true")

### 6.4.3 Learning the hyperparameters

In [ ]:
plots = []
x_train, y_train = create_toy_data(sinusoidal, 7, 0.1, [0.0, 0.7]);
x_train = reshape(x_train, 1, size(x_train)[1])
rbf_kernel = RBFKernel([1.0, 1.0])
model = GaussianProcessRegressor(rbf_kernel, 100);
fitting(model, x_train, y_train)

x = reshape(collect(range(0.0, 1.0, length=100)), 1, 100);
y, y_sigmas = predict(model, x, true)

y_std = sqrt.(reshape(y_sigmas, length(y_sigmas)))
gpr_params = map(x -> round(x, sigdigits=3), rbf_kernel._params)
p = plot(x[1, :], y[1, :], ribbon=(y_std, y_std), color="green", label="std", title="gpr = $(gpr_params)")
p = plot!(x_train[1, :], y_train, seriestype="scatter", label="training")
p = plot!(x[1, :], sin.(2 * pi .* x[1, :]), lw=2, color="red", label="true")
push!(plots, p)

x_train, y_train = create_toy_data(sinusoidal, 7, 0.1, [0.0, 0.7]);
x_train = reshape(x_train, 1, size(x_train)[1])
model = GaussianProcessRegressor(rbf_kernel, 100);
fitting(model, x_train, y_train, 100)

x = reshape(collect(range(0.0, 1.0, length=100)), 1, 100);
y, y_sigmas = predict(model, x, true)

y_std = sqrt.(reshape(y_sigmas, length(y_sigmas)))
gpr_params = map(x -> round(x, sigdigits=3), rbf_kernel._params)
p = plot(x[1, :], y[1, :], ribbon=(y_std, y_std), color="green", label="std", title="gpr = $(gpr_params)")
p = plot!(x_train[1, :], y_train, seriestype="scatter", label="training")
p = plot!(x[1, :], sin.(2 * pi .* x[1, :]), lw=2, color="red", label="true")
push!(plots, p)

plot(plots[1], plots[2], size=(600, 300))

### 6.4.4 Automatic relevance determination

In [ ]:
function create_toy_data_3d(func, n_samples, std)
    x0 = range(0.0, 1.0, length=n_samples)
    x1 = x0 + rand(Normal(0, std), n_samples)
    x2 = rand(Normal(0, std), n_samples)
    t = func(x0) + rand(Normal(0, std), n_samples)
    return vcat(transpose(x0), transpose(x1), transpose(x2)), t
end

x_train, y_train = create_toy_data_3d(sinusoidal, 20, 0.1)
rbf_kernel = RBFKernel([1.0, 1.0, 1.0, 1.0])
model = GaussianProcessRegressor(rbf_kernel, 100)

max_iter = 100
learning_rate = 0.01
updates = [copy(rbf_kernel._params)]
for i in 1:max_iter
    fitting(model, x_train, y_train, 10, learning_rate)
    push!(updates, rbf_kernel._params)
end

params = zeros(size(updates)[1], size(updates[1])[1]);
for i in 1:max_iter
    params[i, :] = updates[i]
end

plot(params[:, 1])
plot!(params[:, 2])
plot!(params[:, 3])